In [20]:
!pip install fasttext

# New section

In [21]:
import numpy as np
import pandas as pd
import tensorflow as tf
import fasttext
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
# from google.colab import files

# uploaded = files.upload()

In [23]:
# Load the training dataset from Excel
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/SC/New_shuffle.xlsx')

# # Load the testing dataset from Excel
tstdf = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/SC/bala 1.xlsx')
#df = pd.read_excel('/content/multichannel_bsentiment_train.xlsx')

# Load the testing dataset from Excel
#tstdf = pd.read_excel('/content/multichannel_bsentiment_test.xlsx')

# Convert 'very positive' to 'positive'
df['Tag'] = df['Tag'].replace('Very Positive', 1)
df['Tag'] = df['Tag'].replace('Positive', 1)
df['Tag'] = df['Tag'].replace('Neutral', 1)
df['Tag'] = df['Tag'].replace('Negative', 3)
df['Tag'] = df['Tag'].replace('Very Negative', 3)
df = df.drop(columns=['No'])

# Convert 'very negative' to 'negative'
tstdf['Tag'] = tstdf['Tag'].replace('Very Positive', 1)
tstdf['Tag'] = tstdf['Tag'].replace('Positive', 1)
tstdf['Tag'] = tstdf['Tag'].replace('Neutral', 1)
tstdf['Tag'] = tstdf['Tag'].replace('Very Negative', 3)
tstdf['Tag'] = tstdf['Tag'].replace('Negative', 3)

display(df)
display(tstdf)

,Comment,Tag
0,এই খুটি তে বাধার নিয়ম ইংরেজ চালু করেছিল তা এখন...,3
1,যারা জনগণের প্রতি দায়বদ্ধ তাদের দায়িত্ববোধ অনে...,1
2,"খেলার জন্যও মানুষ আপনাকে মনে রাখবে, জনগণের জন্...",1
3,বলে কি? সাব্বির কেন?? বাংলাদেশ ক্রিকেটের প্রতি...,3
4,দিন দিন ব‍্যংক এ আমানত কারিদের টাকা র উপর লাভ ...,3
...,...,...
6745,কেউ যদি উনাকে এক প্যাকেট বিস্কুট আর এক কাপ চা ...,3
6746,মান সন্মান না খোয়ালেই চলবে,1
6747,"আার এ জন্যই প্রয়োজন, নির্বাচনকালীন দল নিরপেক্ষ...",1
6748,কেন আপনার নেত্রীতো রাতেই উনার জরুরি সভা ডাকতেন...,3


,No,Comment,Tag
0,1,লিখার সময় পারলে সত্য লিখার অভ্যাস শিখুন।,3
1,2,"এই অর্জন আমাদের গর্ব, আমাদের অহংকার।",1
2,3,তাহলে এ থেকে পাবলিক কী শিক্ষা নিতে পারে!,3
3,4,"সিটিসেলের নাম ,কি আর কইতাম তারা আর কতই নাটক করবে?",3
4,5,শেয়ার প্রতি আয় কমেছে। শূন্য হয় নি এখনও। সরকারী...,3
...,...,...,...
3475,3476,সৈয়দ আশরাফের নির্লিপ্ততা একটা বড় কারন বলে মনে ...,1
3476,3477,আশা করি সরকার আনিসুল হক সাহেবের মত ভাল এক জন ...,1
3477,3478,ভাই সালটা হবে ২০৫০ । এর মধ্যে অনেকগুলি ফ্যাকট...,1
3478,3479,ক্রেডিট কার্ডের সুদের হারঃ-সিটি ব্যাংকের ৩৬%-ই...,1


In [24]:
######
contractions = {
"বি.দ্র ": "বিশেষ দ্রষ্টব্য",
"ড.": "ডক্টর",
"ডা.": "ডাক্তার",
"ইঞ্জি:": "ইঞ্জিনিয়ার",
"রেজি:": "রেজিস্ট্রেশন",
"মি.": "মিস্টার",
"মু.": "মুহাম্মদ",
"মো.": "মোহাম্মদ",
}

!pip3 install googletrans==3.1.0a0
import re
import string
def clean_text(text,remove_stopwords = False):
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    # Format words and remove unwanted characters
    whitespace = re.compile(u"[\s\u0020\u00a0\u1680\u180e\u202f\u205f\u3000\u2000-\u200a]+", re.UNICODE)
    bangla_digits = u"[\u09E6\u09E7\u09E8\u09E9\u09EA\u09EB\u09EC\u09ED\u09EE\u09EF]+"
    english_chars = u"[a-zA-Z0-9]"
    punc = u"[(),$%^&*+={}\[\]:\"|\'\~`<>/,¦!?½£¶¼©⅐⅑⅒⅓⅔⅕⅖⅗⅘⅙⅚⅛⅜⅝⅞⅟↉¤¿º;-]+"
    bangla_fullstop = u"\u0964"     #bangla fullstop(dari)
    punctSeq   = u"['\"“”‘’]+|[.?!,…]+|[:;]+"

    text = re.sub(bangla_digits, " ", text)
    text = re.sub(punc, " ", text)
    text = re.sub(english_chars, " ", text)
    text = re.sub(bangla_fullstop, " ", text)
    text = re.sub(punctSeq, " ", text)
    text = whitespace.sub(" ", text).strip()

    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;‘:‘ ’', '', text)
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]। ,', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    text = re.sub(r"[\@$#%~+-\.\'।\"]"," ",text)
    text = re.sub(r"(?m)^\s+", "", text)
    text = re.sub("[()]","",text)
    text = re.sub("[‘’]","",text)
    text = re.sub("[!]","",text)
    text = re.sub("[/]","",text)
    text = re.sub("[:]","",text)
    text= re.sub('\ |\?|\.|\!|\/|\;|\:', ' ',text)
    text= text.strip("/")

    if remove_stopwords:
        k = []
        with open('gdrive/My Drive/Colab Notebooks/Banglastopword.txt', 'r',encoding="utf-8") as f:
            for word in f:
                word = word.split()
                k.append(word[0])
            text = [t for t in text if t not in k]
            text = "".join(text)

    return text

In [25]:
clean_texts = []
for txt in df.Comment:
    clean_texts.append(clean_text(txt,remove_stopwords=False))
df['Comment'] = clean_texts
display(df)

,Comment,Tag
0,এই খুটি তে বাধার নিয়ম ইংরেজ চালু করেছিল তা এখন...,3
1,যারা জনগণের প্রতি দায়বদ্ধ তাদের দায়িত্ববোধ অনে...,1
2,খেলার জন্যও মানুষ আপনাকে মনে রাখবে জনগণের জন্য...,1
3,বলে কি সাব্বির কেন বাংলাদেশ ক্রিকেটের প্রতি এট...,3
4,দিন দিন ব‍্যংক এ আমানত কারিদের টাকা র উপর লাভ ...,3
...,...,...
6745,কেউ যদি উনাকে এক প্যাকেট বিস্কুট আর এক কাপ চা ...,3
6746,মান সন্মান না খোয়ালেই চলবে,1
6747,আার এ জন্যই প্রয়োজন নির্বাচনকালীন দল নিরপেক্ষ ...,1
6748,কেন আপনার নেত্রীতো রাতেই উনার জরুরি সভা ডাকতেন...,3


In [26]:
X = df['Comment'].values
y = df['Tag'].values
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
# TF-IDF feature extraction
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [28]:
# Print some information about the datasets
print("\nTraining Set Size:", len(X_train))
print("Testing Set Size:", len(X_test))
print("Unique Classes in Training Set:", np.unique(y_train))


Training Set Size: 5400
Testing Set Size: 1350
Unique Classes in Training Set: [1 3]


In [29]:
# Save the TF-IDF data in FastText format
def save_data_fasttext_tfidf(X_tfidf, y, df):
    with open(df, 'w', encoding='utf-8') as f:
        for text, label in zip(X_tfidf, y):
            f.write(f'__label__{label} {text}\n')
save_data_fasttext_tfidf(X_train_tfidf, y_train, 'train_data_tfidf.txt')
save_data_fasttext_tfidf(X_test_tfidf, y_test, 'test_data_tfidf.txt')

In [30]:
# Train the FastText model
model = fasttext.train_supervised(input='train_data_tfidf.txt', epoch=10, lr=0.001, wordNgrams=2)

In [32]:
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
# Obtain probabilities for each class
y_pred_prob = []
for text in X_test:
    prob = model.predict(text)[1][0]
    y_pred_prob.append(prob)

# Convert probabilities to binary predictions using a threshold
threshold = 0.5
y_pred_binary = (np.array(y_pred_prob) > threshold).astype(int)

# Integer encode actual labels for evaluation
label_mapping = {'Negative': 3, 'Positive': 1,'Very Positive':1,'Very Negative':3,'Neutral':1 }
vectorized_mapping = np.vectorize(lambda x: label_mapping.get(x, -1))
y_test_int =vectorized_mapping(y_test)[:len(y_pred_binary)]

#  Calculate precision, recall, and confusion matrix
precision = precision_score(y_test_int, y_pred_binary, average='macro')
recall = recall_score(y_test_int, y_pred_binary, average='macro')
conf_matrix = confusion_matrix(y_test_int, y_pred_binary)

# Print results
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print('Confusion Matrix:')
print(conf_matrix)

# Calculate and print accuracy and classification report
accuracy = accuracy_score(y_test_int, y_pred_binary)
class_report = classification_report(y_test_int, y_pred_binary)
print(f'Accuracy: {accuracy:.4f}')

print(len(y_test_int))
print(len(y_pred_binary))
print(type(y_test_int))
print(type(y_pred_binary))

# If needed, you can print the actual values
print("y_test_int:", y_test_int)
print("y_pred_binary:", y_pred_binary)


Precision: 0.0000
Recall: 0.0000
Confusion Matrix:
[[   0 1350]
 [   0    0]]
Accuracy: 0.0000
1350
1350
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
y_test_int: [-1 -1 -1 ... -1 -1 -1]
y_pred_binary: [1 1 1 ... 1 1 1]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWa